<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
import re

In [7]:
df = yelp

In [8]:
def tokenize(doc):
    doc = nlp(doc)
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [9]:
df['tokens'] = df['text'].apply(tokenize)
df['tokens'].head(3)

0    [beware, fake, FAKE, fake, small, business, Lo...
1    [come, lunch, Togo, service, quick, staff, fri...
2    [Vegas, dozen, time, step, foot, Circus, Circu...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [11]:
# Classics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [12]:
texts = df['text'].tolist()

In [13]:
stars = df['stars'].tolist()

In [14]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                        tokenizer=tokenize,
                        ngram_range=(1,2),
                        min_df=3,
                        max_df=0.25)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(df['text'])

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
print(dtm.shape)
dtm.head()

(10000, 32713)


,,$,+,-PRON-,-duck,-i,-it,-the,-they,1,...,à,à ce,à essayer,à la,ça,équipe,était,été,être,‍
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
og_df = df

In [16]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# Calculate Distance of TF-IDF Vectors
dist_matrix = cosine_similarity(dtm)
print(dist_matrix.shape)

# Turn it into a DataFrame
matrix = pd.DataFrame(dist_matrix)
matrix.head()

(10000, 10000)


,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,1.000000,0.000000,0.019832,0.000000,0.005758,0.012170,0.014579,0.000000,0.008339,0.015154,...,0.000000,0.000000,0.009610,0.016761,0.006772,0.034497,0.000000,0.005112,0.002811,0.016703
1,0.000000,1.000000,0.021155,0.000000,0.020723,0.014462,0.000000,0.000000,0.000000,0.000000,...,0.008553,0.010277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003244
2,0.019832,0.021155,1.000000,0.011055,0.011742,0.005327,0.015274,0.000000,0.040137,0.019198,...,0.019595,0.024657,0.009447,0.023913,0.012910,0.023076,0.002196,0.003462,0.007314,0.065126
3,0.000000,0.000000,0.011055,1.000000,0.008660,0.029398,0.009750,0.000000,0.007209,0.000000,...,0.000000,0.000000,0.009925,0.000000,0.000000,0.079092,0.000000,0.000000,0.000000,0.054971
4,0.005758,0.020723,0.011742,0.008660,1.000000,0.018284,0.000000,0.017498,0.012418,0.043243,...,0.006551,0.005064,0.031218,0.000000,0.062136,0.000000,0.010864,0.014155,0.003264,0.031582


In [17]:
from sklearn.neighbors import NearestNeighbors

# Fit on DTM
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

# sample a doc from dtm to use as our query point 
doc = dtm.iloc[0].values

# Query Using kneighbors 
nn.kneighbors([doc])

(array([[0.        , 1.        , 1.        , 1.24040014, 1.2817619 ,
         1.28256456, 1.30552944, 1.31768906, 1.31973027, 1.32710404]]),
 array([[   0, 6204, 6311, 2131, 3276, 6132, 7063, 9036, 2943, 6019]]))

In [18]:
# Create a fake review and find the 10 most similar reviews

fake_review = ['''Beautiful new location for Maman near NYU! The outdoor floral decorations are stunning. We sat next to the French windows that I believe open to a porch balcony vibe in the warmer months.

The staff was very good about keeping everyone socially distanced and taking peoples temperatures and contact information if contact tracing was needed.

My friend and I shared some cookies and baked pastries with the famous Lavendar hot chocolate and earl grey tea.

Great service will come again!''']

fake_review_vect = tfidf.transform(fake_review)
fake_review_vect

<1x32713 sparse matrix of type '<class 'numpy.float64'>'
	with 49 stored elements in Compressed Sparse Row format>

In [19]:
review_nn = nn.kneighbors(fake_review_vect.todense())
review_nn

(array([[1.        , 1.        , 1.30421524, 1.31571116, 1.31668318,
         1.31751497, 1.32518252, 1.3253673 , 1.32891846, 1.32982072]]),
 array([[6311, 6204, 6132, 4977, 2779, 4282, 1595, 4720, 3531, 5241]]))

In [20]:
for review_num in review_nn[1][0]:
    print(review_num, texts[review_num])
    print()

6311 天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來

6204 旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました。
予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをしてもらい、とても気に入りました。
また是非マッサージなどで伺いたいと思います。

6132 Hi, 
Update: I went to the location and spoke to the people around, they think there is something fishy going on...if someone is able to get some information, please contact me...my email id is akshayinfy@gmail.com

i am not able to contact these people since the last  few days, they have had my car for a while and are not replying to phone calls or emails...i see others also having this problem, does anyone have any contact numbers of any staff member? Someone named Brian owns the place...does anybody have any phone number for this person?????

4977 Very cute and cozy little coffee shop/bakery. They even have gelato. Prices are very reasonable and the two gentlemen

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [22]:
from sklearn.svm import LinearSVC

# Create Pipeline Components
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),# result lies in 1 word or 2 words
                       min_df=2,
                       max_df=0.25)
svm = LinearSVC()

In [23]:
# Define the Pipeline
pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('clf', svm)         # clf: classifier, LinearSVC Classifier
])
pipe

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.25, max_features=None,
                                 min_df=2, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
          

In [24]:
parameters = {
    'vect__max_df': (0.3, 0.5),
    'clf__C': (0.5, 1.)
}

gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=1, verbose=1)

In [25]:
gs.fit(texts, stars)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:   54.6s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.25,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 2),
                                                        n

In [26]:
gs.best_score_

0.6188

In [27]:
gs.best_params_

{'clf__C': 0.5, 'vect__max_df': 0.5}

In [28]:
pred = gs.predict(fake_review)

In [29]:
pred

array([5])

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [30]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [32]:
id2word = Dictionary(df['tokens'])

Create a bag of words representation of the entire corpus

In [33]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

Your LDA model should be ready for estimation: 

In [34]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 5 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim 

# Use pyLDAvis (or a ploting tool of your choice) to visualize your results 

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.005418  0.001155       1        1  20.884530
2     -0.001124 -0.000450       2        1  20.274713
4      0.003541  0.002388       3        1  19.830083
3      0.001229 -0.005125       4        1  19.721997
1      0.001771  0.002031       5        1  19.288676, topic_info=           Term          Freq         Total Category  logprob  loglift
0                26347.000000  26347.000000  Default  30.0000  30.0000
369       order   3280.000000   3280.000000  Default  29.0000  29.0000
47         good   6055.000000   6055.000000  Default  28.0000  28.0000
41          try   2458.000000   2458.000000  Default  27.0000  27.0000
52      service   3527.000000   3527.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
339  definitely    286.891112   1389.415475   Topic5  -5.8587   0.0681
164       think    287.288591   1420.248652   Topic5  -5.8573   0.0476
1             $    372.503321   2160.140591   Topic5  -5.5975  -0.1120
41          try    400.458228   2458.798052   Topic5  -5.5252  -0.1692
369       order    363.013314   3280.232078   Topic5  -5.6233  -0.5556

[461 rows x 6 columns], token_table=       Topic      Freq                                               Term
term                                                                     
0          1  0.267840                                                   
0          2  0.226812                                                   
0          3  0.158913                                                   
0          4  0.180774                                                   
0          5  0.165706                                                   
...      ...       ...                                                ...
22970      1  0.622513                 ネイルはちはるさんと言う綺麗な方が丁寧にしてくれとても気に入りました
22971      1  0.622744  予定になかったまつ毛エクステもお願いし、日本ではまだあまりないブラウンカラーのエクステをして...
23143      2  0.621222  天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
22972      1  0.622327              旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした
22973      1  0.622895  質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のい...

[1336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 5, 4, 2])

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Create more visualizations of the LDA results and provide written analysis
* Incorporate RandomizedSearchCV into docoument classification pipeline
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)